# Introduction

Notebook para avaliação de NAS-v3 com dataset NATS-Bench-201.

# Import Libraries

In [1]:
import os
# disable tensorflow log level infos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

In [2]:
import sys

if '../..' not in sys.path:
    sys.path.insert(0, '../..')

In [3]:
import os
import time
import random
import nats_bench

import numpy as np
import pyglove as pg
import tensorflow as tf

from absl import app
from absl import flags

from typing import Tuple

# Check GPU Availability

In [4]:
## restrict memory growth -------------------
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try:
    gpu_0 = physical_devices[0]
    tf.config.experimental.set_memory_growth(gpu_0, True) 
    #tf.config.experimental.set_virtual_device_configuration(gpu_0, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6500)])
    print(' ==> Restrict GPU memory growth: True')
except: 
    raise Exception("Invalid device or cannot modify virtual devices once initialized.")
## restrict memory growth ------------------- 

 ==> Restrict GPU memory growth: True


# NAS

## NASController_4

In [72]:
import os
import numpy as np

import tensorflow.keras.backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, RNN, LSTMCell, Input
from tensorflow.keras.preprocessing.sequence import pad_sequences


from src.base.experiment.training.optimizers import Optimizer
from src.nas.v3.mlp_search_space import MLPSearchSpaceCandidate

class NASController_4:
    def __init__(self, config_interp):        
        self.config_interp = config_interp
        
        self.controller_classes         = self.config_interp.controller_params['controller_classes']
        self.controller_lstm_dim        = self.config_interp.controller_params['controller_lstm_dim']
        self.controller_optimizer       = self.config_interp.controller_params['controller_optimizer']
        self.controller_lr              = self.config_interp.controller_params['controller_learning_rate']
        self.controller_decay           = self.config_interp.controller_params['controller_decay']
        self.controller_momentum        = self.config_interp.controller_params['controller_momentum']
        self.use_predictor              = self.config_interp.controller_params['controller_use_predictor']
        self.controller_loss_alpha      = self.config_interp.controller_params['controller_loss_alpha']
        self.controller_train_epochs    = self.config_interp.controller_params['controller_training_epochs']
        self.controller_sampling_epochs = self.config_interp.controller_params['controller_sampling_epochs']
        
        self.max_proposed_arch_length = self.config_interp.nas_params['max_proposed_arch_length']
        
        self.controller_weights_path = 'LOGS/controller_weights.h5'

        self.n_tasks = len(self.config_interp.tasks)
        
        self.controller_noise_dim = 4
        self.controller_input_shape = (1, self.controller_classes+self.controller_noise_dim)
        
        self.controller_batch_size = self.config_interp.controller_params['controller_batch_size']

        self.controller_use_predictor = self.config_interp.controller_params['controller_use_predictor']
        
        self.nas_data_history = None
        self.search_space_candidates = MLPSearchSpaceCandidate.N_DENSES.value + MLPSearchSpaceCandidate.N_CONVS.value
        self.search_space_candidates_size = len(self.search_space_candidates)

        self.__clean_controller_weights()        

        if not self.controller_use_predictor:
            self.controller_model = self.__create_control_model()
        else:
            self.controller_model = self.__create_hybrid_control_model()


    def __clean_controller_weights(self):
        if os.path.exists(self.controller_weights_path):
            os.remove(self.controller_weights_path)


    def __check_nas_data_history(self, new_dna_value):
        if self.nas_data_history is None:
            return True
        else:
            for tup in self.nas_data_history:
                existing_dna = tup[0].to_numbers()
                new_dna_value_rep = list(new_dna_value[0][0])
                #print(f' ..{existing_dna}, {new_dna_value_rep}')
                if np.array_equal(existing_dna, new_dna_value_rep):
                    #print(f' ...Invalid DNA -> existing_dna: {existing_dna} | new_dna_value: {new_dna_value_rep}')
                    #print(f' ....Proposing new architecture!')
                    return False
            #print(f' ..New valid DNA: {new_dna_value}')
            return True
    
            
    def build_new_arch(self, prev_arch):
        inp = np.array(prev_arch).reshape(1,1,self.max_proposed_arch_length)

        # print(f' ..input: {inp}')

        final_arch = []
        for i in range(self.max_proposed_arch_length):
            noise = np.random.randint(0, 10, size=4).reshape(1,1,4)
            # print(f' ..noise: {noise}')

            inp = np.concatenate([inp,noise], axis=2)

            prob_list = self.controller_model.predict(inp)   # output: (1,1,n_classes+noise_dim)
            prob_list = prob_list[0][0]
            
            chose_idx = np.argmax(prob_list)  # choose from the first part of the list (5 classes)
            
            final_arch.append(int(chose_idx))
            # print(f'final_arch: {final_arch}')

            new_arch = pad_sequences([final_arch], maxlen=self.max_proposed_arch_length, padding='post', value=-1)
            # print(new_arch.shape)

            inp = np.array(new_arch).reshape(1,1,self.max_proposed_arch_length)
            # print(f' ..new input: {inp}')
        
        final_arch = [[final_arch]]
        # print(f' .final_arch: {final_arch}')
        
        # if not self.__check_nas_data_history(prev_arch):
        #     print(' ..redoing proposal')
        #     self.build_new_arch(prev_arch)   # redo the process if architecture already exists

        return final_arch


    def __prepare_controller_data(self, nas_data_history):
        self.nas_data_history = nas_data_history

        # print('Preparing controller data...')
        # print(f' ..nas_data_history: {nas_data_history}')
        
        xc = np.array([[item[0].to_numbers() for item in nas_data_history]])       
        xc = xc.reshape(self.controller_batch_size, 1, self.controller_classes)
        # print(f' ..xc.shape: {xc.shape}')

        noise = np.random.randint(0, 10, size=(self.controller_batch_size, 1, self.controller_noise_dim))
        xc = np.concatenate([xc, noise], axis=2)
        # print(f' ..xc + noise.shape: {xc.shape}')

        #yc = to_categorical(controller_sequences[:, -1], self.controller_classes)
        yc = np.zeros((self.controller_batch_size, 1, self.controller_classes))
        # print(f' ..yc.shape: {yc.shape}')
        
        val_acc_target = [item[1] for item in nas_data_history]
        # print(f' ..val_acc_target.length: {len(val_acc_target)}')
        
        # print(' ..done!')
        
        return xc, yc, val_acc_target


    def get_discounted_reward(self, rewards):
        # initialise discounted reward array
        discounted_r = np.zeros_like(rewards, dtype=np.float32)

        # every element in the discounted reward array
        for t in range(len(rewards)):
            running_add = 0.
            exp = 0.

            # will need us to iterate over all rewards from t to T
            for r in rewards[t:]:
                running_add += self.controller_loss_alpha**exp * r
                exp += 1
            
            # add values to the discounted reward array
            discounted_r[t] = running_add

        # normalize discounted reward array    
        discounted_r = (discounted_r - discounted_r.mean()) / discounted_r.std()

        return discounted_r


    # loss function based on discounted reward for policy gradients
    def custom_loss(self, target, output):
        # define baseline for rewards and subtract it from all validation accuracies to get reward.
        baseline = 0.5
        reward = np.array([item[1] - baseline for item in self.nas_data_history[-self.controller_batch_size:]]).reshape(
           self.controller_batch_size, 1)
        
        # get discounted reward
        discounted_reward = self.get_discounted_reward(reward)

        # multiply discounted reward by log likelihood of actions to get loss function
        loss = - K.log(output) * discounted_reward[:, None]

        return loss


    def train_model_controller(self, nas_data_history):
        # print(' ..Training controller model...')

        nas_data_history = nas_data_history[-self.controller_batch_size:]

        xc, yc, val_acc_target = self.__prepare_controller_data(nas_data_history)

        if self.use_predictor:
            self.__train_hybrid_control_model(
                                            xc,
                                            yc,
                                            val_acc_target,
                                            self.custom_loss,
                                            self.controller_batch_size,
                                            self.controller_train_epochs)
        else:
            self.__train_control_model(
                                     xc,
                                     yc,
                                     self.custom_loss,
                                     self.controller_batch_size,
                                     self.controller_train_epochs)
        
        # print(' ..Controller model trained!')


    def __get_optimizer(self):
        if self.controller_optimizer.name == Optimizer.SGD.name:
            optim = optimizers.SGD(learning_rate=self.controller_lr, decay=self.controller_decay, momentum=self.controller_momentum, clipnorm=1.0)
        elif self.controller_optimizer.name == Optimizer.SGD_NESTEROV.name:
            optim = optimizers.SGD(learning_rate=self.controller_lr, decay=self.controller_decay, momentum=self.controller_momentum, nesterov=True, clipnorm=1.0)
        else:
            optim = getattr(optimizers, self.controller_optimizer.value)(learning_rate=self.controller_lr, decay=self.controller_decay, clipnorm=1.0)
        return optim


    def __create_control_model(self):
        main_input = Input(shape=self.controller_input_shape, name='main_input')        
        # print(f'Controller model input shape: {main_input.shape}')
        x = RNN(LSTMCell(self.controller_lstm_dim), return_sequences=True)(main_input)
        main_output = Dense(self.controller_classes, activation='softmax', name='main_output')(x)
        # print(f'Controller model output shape: {main_output.shape}')
        model = Model(inputs=[main_input], outputs=[main_output])
        return model


    def __train_control_model(self, x_data, y_data, loss_func, controller_batch_size, nb_epochs):
        optim = self.__get_optimizer()
        
        self.controller_model.compile(optimizer=optim, loss={'main_output': loss_func})
        
        if os.path.exists(self.controller_weights_path):
            self.controller_model.load_weights(self.controller_weights_path)
        
        # print("TRAINING CONTROLLER...")
        
        self.controller_model.fit({'main_input': x_data},
                  {'main_output': y_data.reshape(len(y_data), 1, self.controller_classes)},
                  epochs=nb_epochs,
                  batch_size=controller_batch_size,
                  verbose=0)
        
        self.controller_model.save_weights(self.controller_weights_path)


    # ------------------- Hybrid Model -------------------

    
    def __create_hybrid_control_model(self):
        main_input = Input(shape=self.controller_input_shape, name='main_input')
        x = RNN(LSTMCell(self.controller_lstm_dim), return_sequences=True)(main_input)
        predictor_output = Dense(1, activation='sigmoid', name='predictor_output')(x)
        main_output = Dense(self.controller_classes, activation='softmax', name='main_output')(x)
        model = Model(inputs=[main_input], outputs=[main_output, predictor_output])
        return model


    def __train_hybrid_control_model(self, x_data, y_data, pred_target, loss_func, controller_batch_size, nb_epochs):

        optim = self.__get_optimizer()
        
        self.controller_model.compile(optimizer=optim,
                      loss={'main_output': loss_func, 'predictor_output': 'mse'},
                      loss_weights={'main_output': 1, 'predictor_output': 1})
        
        if os.path.exists(self.controller_weights_path):
            self.controller_model.load_weights(self.controller_weights_path)
        
        print("TRAINING CONTROLLER...")
        
        self.controller_model.fit({'main_input': x_data},
                  {'main_output': y_data.reshape(len(y_data), 1, self.controller_classes),
                   'predictor_output': np.array(pred_target).reshape(len(pred_target), 1, 1)},
                  epochs=nb_epochs,
                  batch_size=controller_batch_size,
                  verbose=0)
        
        self.controller_model.save_weights(self.controller_weights_path)

    
    def get_predicted_accuracies_hybrid_model(self, model, seqs):
        pred_accuracies = []
        for seq in seqs:
            control_sequences = pad_sequences([seq], maxlen=self.controller_classes, padding='post')
            xc = control_sequences[:, :-1].reshape(len(control_sequences), 1, self.controller_classes - 1)
            (_, pred_accuracy) = [x[0][0] for x in model.predict(xc)]
            pred_accuracies.append(pred_accuracy[0])
        return pred_accuracies

## ConfigInterp

In [73]:
import yaml
import pprint

from typing import List

from src.m_utils.nas_mtl_approach import NAS_MTLApproach
from src.m_utils.mtl_approach import MTLApproach
from src.m_utils.stl_approach import STLApproach
from src.m_utils.utils import print_method_log_sig
from src.base.experiment.dataset.dataset import Dataset
from src.base.experiment.tasks.task import TASK

class ConfigInterpreter:
    def __init__(self, kwargs, yaml_config_file=None):
        if yaml_config_file != None:
            kwargs = self.__load_exp_config(yaml_config_file)

        self.use_neptune = kwargs['use_neptune']
        print('-----')
        print('Use Neptune: ', self.use_neptune)
        print('-----')
        
        print('-------------------')
        print('Args: ')
        pprint.pprint(kwargs)
        print('-------------------')
        
        self.exp_args = kwargs['exp_params']
        self.prop_args = kwargs['properties']
        
        self.mlp_params = kwargs['mlp_params']

        self.nas_params = kwargs['nas_params']
        self.controller_params = kwargs['controller_params']
        
        self.__kwargs_sanity_check()

        self.dataset : Dataset = self.prop_args['dataset']
        self.tasks : List[TASK] = self.prop_args['tasks']

        self.base_model = self.mlp_params['mlp_base_model']
        print('----')
        print('Base Model Name: ', self.base_model)
        print('----')

        self.is_mtl_model = self.__get_is_mtl_model()
        self.approach = self.__get_approach()
        self.is_nas_mtl_model = self.__get_is_nas_mtl_model()
        self.exec_nas = self.prop_args['exec_nas']
        
        print('----')


    def __get_approach(self):
        approach = self.prop_args['approach']

        if len(self.tasks) == 1:
            assert isinstance(approach, STLApproach)
        elif len(self.tasks) > 1:
            assert isinstance(approach, MTLApproach) or isinstance(approach, NAS_MTLApproach)
        else:
            raise Exception('Invalid approach')

        print(f'Approach: {approach}')

        return approach
    

    def __get_is_mtl_model(self):
        is_mtl_model = len(self.tasks) > 1
        print(f'MTL Model: {is_mtl_model}')
        return is_mtl_model
    

    def __get_is_nas_mtl_model(self):
        is_nas_mtl_model = len(self.tasks) > 1 and self.prop_args['exec_nas']
        print(f'NAS MTL Model: {is_nas_mtl_model}')
        return is_nas_mtl_model


    def __load_exp_config(self, yaml_config_file):
        print_method_log_sig('load experiment configs')
        print(f'Loading experiment config from {yaml_config_file}')
        with open(yaml_config_file, 'r') as f:
            cnt = yaml.load(f, Loader=yaml.Loader)[0]
            print('..Experiment configs loaded with success!')
            return cnt
    

    def __kwargs_sanity_check(self):
        has_experiment_id = True if self.prop_args['orig_model_experiment_id'] != '' else False
        is_training_new_model = self.prop_args['train_model']
        
        if not has_experiment_id and not is_training_new_model:
            raise Exception('You must train a new model or provide an experiment ID')        

In [74]:
%%capture

from src.nas.v3.nas_algorithm import NASAlgorithm
from src.nas.v3.mlp_search_space import MLPSearchSpaceIndicator
from src.base.experiment.training.optimizers import Optimizer
from src.base.experiment.training.base_models import BaseModel


DATASET = Dataset.MNIST
APPROACH = NAS_MTLApproach.APPROACH_3

kwargs = { 
    'use_neptune': False,
    'exp_params' : {
        'name': 'NAS experiment',
        'description': 'NAS with Approach 3 - Testing parametrization of n_convs',
        'tags': [f'{DATASET.value["name"]}', 'nas', 'nas_approach_3_v2', 'no_use_predictor'],
        'src_files': ["../src/**/*.py"]
    },
    'properties': {
        'approach': APPROACH,
        'dataset': DATASET,
        'tasks': DATASET.value['tasks'],
        'balance_input_data': False,
        'train_model': True,
        'save_trained_model': True,
        'exec_nas': True,
        'orig_model_experiment_id': '',
        'sample_training_data': True,
        'sample_prop': 0.02
    },
    'nas_params': {
        'architecture_training_epochs': 2,     # n_epochs for training proposed architecture
        'max_proposed_arch_length': 5,
        'total_num_proposed_architectures': 10,
        'nas_algorithm': NASAlgorithm.RL,
        'nas_search_space': MLPSearchSpaceIndicator.SS_2
    },
    'controller_params': {
        'controller_classes': 7,
        'controller_lstm_dim': 100,
        'controller_optimizer': Optimizer.ADAM,
        'controller_learning_rate': 0.01,
        'controller_decay': 0.1,
        'controller_momentum': 0.0,
        'controller_use_predictor': False,
        'controller_loss_alpha': 0.9,
        'controller_training_epochs': 5,
        'controller_sampling_epochs': 6,
        'controller_batch_size': 64
    },
    'mlp_params': {
        'mlp_base_model': BaseModel.MOBILENET_V2,
        'mlp_n_epochs': 50,
        'mlp_batch_size': 64,
        'mlp_early_stopping': 50,
        'mlp_optimizer': Optimizer.ADAMAX,
        'mlp_learning_rate': 1e-3,
        'mlp_decay': 0.0,
        'mlp_momentum': 0.0,
        'mlp_dropout': 0.3,
        'mlp_loss_function': 'sparse_categorical_crossentropy',
        'mlp_one_shot': False
    }
}

config_interp = ConfigInterpreter(kwargs)

## RL_DNAGenerator

In [75]:
import random
import pyglove as pg
import numpy as np
import pandas as pd


class RL_DNAGenerator(pg.generators.geno.DNAGenerator):
    def __init__(self, config_interp: ConfigInterpreter):
        self.nas_history_data : list = []
        self.nas_controller: NASController_4 = NASController_4(config_interp)
        self.nas_data_log_path = 'LOGS/nas_data.csv'


    def _feedback(self, dna, reward):
        self.nas_history_data.append([dna,reward])
        
        #print(f' ..nas_history_data: {self.nas_history_data}')
        #print(f' ..len(self.nas_history_data): {len(self.nas_history_data)}')

        self.__log_data()

    
    def __log_data(self):
        # print(f' ..Logging nas_history_data...')
        # print(f' ..self.nas_history_data: {self.nas_history_data}')
        # print(f' ..nas_data_log_path: {self.nas_data_log_path}')
        df = pd.DataFrame(data=self.nas_history_data, columns=['dna', 'reward'])
        df.to_csv(self.nas_data_log_path, index=False)
        # print(' ..done!')


    def _propose(self):
        if self.num_feedbacks % self.nas_controller.controller_batch_size == 0 and self.num_feedbacks > 0:
            # print(70*'.')
            # print(' ..New batch of architectures. Training controller model...')
            self.nas_controller.train_model_controller(self.nas_history_data)    
            # print(70*'.')
        
        prev_arch = None
        if self.num_feedbacks > 0:
            prev_arch = self.nas_history_data[-1][0].to_numbers()
        else:
            prev_arch = [random.randint(0,10) for _ in range(self.nas_controller.max_proposed_arch_length)]

        prev_arch = np.array(prev_arch).reshape(1,1,self.nas_controller.max_proposed_arch_length)

        # print(f' ..prev_arch: {prev_arch}')
        # print(f' ..prev_arch.shape: {prev_arch.shape}')

        new_arch = self.nas_controller.build_new_arch(prev_arch)
        
        return pg.geno.DNA(new_arch)

## Utilitary Functions

In [76]:
DEFAULT_NATS_FILEs = dict(tss=None, sss=None)
DEFAULT_REPORTING_EPOCH = dict(tss=200, sss=90)
VALIDATION_SET_REPORTING_EPOCH = 12


@pg.functor([('channels', pg.typing.List(pg.typing.Int()))])
def model_sss_spc(channels):
    return ':'.join(str(x) for x in channels)


def get_search_space(ss_indicator):
    info = nats_bench.search_space_info('nats-bench', ss_indicator)
    print(f'Candidates: {info["candidates"]}')
    if ss_indicator == 'sss':
        return model_sss_spc(pg.sublist_of(info['num_layers'], info['candidates'], choices_distinct=False))

    
def get_algorithm(algorithm_str):
    """Creates algorithm."""
    if algorithm_str == 'random':
        return pg.generators.Random()
    elif algorithm_str == 'evolution':
        return pg.evolution.regularized_evolution(mutator=pg.evolution.mutators.Uniform(), population_size=50, tournament_size=10)
    elif algorithm_str == 'rl':
        return RL_DNAGenerator(config_interp)
    else:
        return pg.load(algorithm_str)
    

def search(nats_api, search_model, algo, dataset='cifar10', reporting_epoch=12, max_train_hours=2e4):
    nats_api.reset_time()
    valid_models = 0
    time_spent_in_secs = 0
    start_time = time.time()
    last_report_time = start_time
    
    results_df = pd.DataFrame(columns=['id','dna','cell_spec',
                                       'val_acc','latency','time_cost','total_time',
                                       'test_acc','test_loss','test_per_time','test_all_time',
                                       'time_spent_in_hours','time_spent_in_secs',
                                       'train_accuracy','train_loss','train_per_time','train_all_time',
                                       'comment'])
    
    for model, feedback in pg.sample(search_model, algo):
        spec = model()
        
        (validation_accuracy, latency, time_cost, total_time) = nats_api.simulate_train_eval(spec, dataset=dataset, hp=VALIDATION_SET_REPORTING_EPOCH)
        
        time_spent_in_secs = nats_api.used_time
        
        more_info = nats_api.get_more_info(spec, dataset, hp=reporting_epoch)  # pytype: disable=wrong-arg-types  # dict-kwargs
        
        valid_models += 1
        
        feedback(validation_accuracy)
        
        time_spent_in_hours = time_spent_in_secs / (60 * 60)
        
        if time_spent_in_hours > max_train_hours:
            break # Break the first time we exceed the budget.
        
        if feedback.id % 100 == 0:
            now = time.time()
            print(f'Tried {feedback.id} models, valid {valid_models}, '
                  f'time_spent_in_hours: {int(time_spent_in_hours)}h, '
                  f'time_spent_in_secs: {round(time_spent_in_secs,3)}s, '
                  f'elapse since last report: {round(now - last_report_time,3)}s.')
            last_report_time = now
        
        formatted_spec = ':'.join(['{:02d}'.format(int(x)) for x in spec.split(':')])
        #print(f'Cell-spec: {formatted_spec} | ID: {feedback.id} | DNA: {feedback.dna} | Validation Acc: {validation_accuracy}')
        
        results_df.loc[len(results_df)] = {'id': feedback.id,
                                           'cell_spec': formatted_spec, 
                                           'dna': feedback.dna, 
                                           'val_acc': validation_accuracy, 
                                           'latency': latency,
                                           'time_cost': time_cost,
                                           'total_time': total_time,
                                           'test_acc': more_info['test-accuracy'],
                                           'test_loss': more_info['test-loss'],
                                           'test_per_time': more_info['test-per-time'],
                                           'test_all_time': more_info['test-all-time'],
                                           'time_spent_in_secs': round(time_spent_in_secs,3),
                                           'time_spent_in_hours': int(time_spent_in_hours),
                                           'train_loss': more_info['train-loss'],
                                           'train_accuracy': more_info['train-accuracy'],
                                           'train_per_time': more_info['train-per-time'],
                                           'train_all_time': more_info['train-all-time'],
                                           'comment': more_info['comment']}
        
            
    print(f'Total time elapse: {time.time() - start_time} seconds.')
    
    return results_df
    
    
def test_nas_algo(algo_name):
    SEARCH_SPACE = 'sss'    
    
    nats_bench.api_utils.reset_file_system('default')
    nats_api = nats_bench.create(DEFAULT_NATS_FILEs[SEARCH_SPACE], SEARCH_SPACE, fast_mode=True, verbose=False)

    search_model = get_search_space(SEARCH_SPACE)
    reporting_epoch = DEFAULT_REPORTING_EPOCH[SEARCH_SPACE]

    algorithm = get_algorithm(algo_name)

    results_df = search(nats_api, search_model, algorithm, 'cifar10', reporting_epoch, max_train_hours=3)
    
    sorted_results = results_df.sort_values(by='val_acc', ascending=False)
    
    return sorted_results
    
    
def print_report(sorted_results_df):
    n_trials = len(sorted_results_df)
    print(f'Best architecture found after {n_trials} evaluated models!')
    print('Best model found: ')
    display(sorted_results_df.iloc[:,:13].head(1))

# Tests NAS ALgorithms

## Test with Random Algorithm

In [28]:
sorted_results_rnd = test_nas_algo('random')
sorted_results_rnd.head()

[2023-06-28 00:45:16] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.
Candidates: [8, 16, 24, 32, 40, 48, 56, 64]
Tried 100 models, valid 100, time_spent_in_hours: 1h, time_spent_in_secs: 5173.49s, elapse since last report: 1.801s.
Tried 200 models, valid 200, time_spent_in_hours: 2h, time_spent_in_secs: 10469.77s, elapse since last report: 1.82s.
Tried 300 models, valid 300, time_spent_in_hours: 4h, time_spent_in_secs: 15685.205s, elapse since last report: 1.835s.
Tried 400 models, valid 400, time_spent_in_hours: 5h, time_spent_in_secs: 20946.303s, elapse since last report: 1.848s.
Tried 500 models, valid 500, time_spent_in_hours: 7h, time_spent_in_secs: 26271.46s, elapse since last report: 1.847s.
Tried 600 models, valid 600, time_spent_in_hours: 8h, time_spent_in_secs: 31490.496s, elapse since last report: 1.788s.
Tried 700 models, valid 700, time_spent_in_hours: 10h, time_spent_in_secs: 36832.76s, elapse since last report: 2.441s.
Tried 800 models, 

,id,dna,cell_spec,val_acc,latency,time_cost,total_time,test_acc,test_loss,test_per_time,test_all_time,time_spent_in_hours,time_spent_in_secs,train_accuracy,train_loss,train_per_time,train_all_time,comment
2645,2646,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.020,0.020324,69.808378,138888.197737,93.18,0.261427,0.860657,77.459171,38,138888.198,99.910,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."
3336,3337,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.020,0.020324,69.808378,175693.649063,93.18,0.261427,0.860657,77.459171,48,175693.649,99.910,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."
3794,3795,"DNA([7, 7, 7, 7, 5])",64:64:64:64:48,84.672,0.020464,70.513710,199747.001945,93.45,0.251412,0.845815,76.123366,55,199747.002,99.924,0.007676,10.850560,976.550417,"In this dict, train-loss/accuracy/time is the ..."
2509,2510,"DNA([7, 7, 6, 7, 6])",64:64:56:64:56,84.612,0.020573,71.375285,131691.330401,93.30,0.255049,0.859325,77.339222,36,131691.330,99.874,0.008999,11.038129,993.431597,"In this dict, train-loss/accuracy/time is the ..."
6581,6582,"DNA([7, 6, 7, 7, 4])",64:56:64:64:40,84.576,0.020175,70.785582,346486.535568,92.96,0.279243,0.866001,77.940080,96,346486.536,99.868,0.009217,10.970484,987.343562,"In this dict, train-loss/accuracy/time is the ..."


In [29]:
print_report(sorted_results_rnd)

,id,dna,cell_spec,val_acc,latency,time_cost,total_time,test_acc,test_loss,test_per_time,test_all_time,time_spent_in_hours,time_spent_in_secs
2645,2646,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,138888.197737,93.18,0.261427,0.860657,77.459171,38,138888.198


In [30]:
sorted_results_rnd.to_csv('data/random_100h.csv', index=False)

## Test with Regularized Evolution Algorithm

In [31]:
sorted_results_evol = test_nas_algo('evolution')
sorted_results_evol.head()

[2023-06-28 00:49:16] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.
Candidates: [8, 16, 24, 32, 40, 48, 56, 64]
Tried 100 models, valid 100, time_spent_in_hours: 1h, time_spent_in_secs: 5593.314s, elapse since last report: 1.766s.
Tried 200 models, valid 200, time_spent_in_hours: 3h, time_spent_in_secs: 11795.439s, elapse since last report: 1.605s.
Tried 300 models, valid 300, time_spent_in_hours: 5h, time_spent_in_secs: 18231.932s, elapse since last report: 1.51s.
Tried 400 models, valid 400, time_spent_in_hours: 6h, time_spent_in_secs: 24663.906s, elapse since last report: 2.621s.
Tried 500 models, valid 500, time_spent_in_hours: 8h, time_spent_in_secs: 31288.741s, elapse since last report: 1.518s.
Tried 600 models, valid 600, time_spent_in_hours: 10h, time_spent_in_secs: 37791.194s, elapse since last report: 1.421s.
Tried 700 models, valid 700, time_spent_in_hours: 12h, time_spent_in_secs: 44360.279s, elapse since last report: 1.45s.
Tried 800 mode

,id,dna,cell_spec,val_acc,latency,time_cost,total_time,test_acc,test_loss,test_per_time,test_all_time,time_spent_in_hours,time_spent_in_secs,train_accuracy,train_loss,train_per_time,train_all_time,comment
5068,5069,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,323456.034085,93.18,0.261427,0.860657,77.459171,89,323456.034,99.91,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."
2611,2612,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,166808.332951,93.18,0.261427,0.860657,77.459171,46,166808.333,99.91,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."
1286,1287,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,82184.983194,93.18,0.261427,0.860657,77.459171,22,82184.983,99.91,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."
2608,2609,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,166601.095250,93.18,0.261427,0.860657,77.459171,46,166601.095,99.91,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."
5001,5002,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,319136.337607,93.18,0.261427,0.860657,77.459171,88,319136.338,99.91,0.007389,10.772471,969.522386,"In this dict, train-loss/accuracy/time is the ..."


In [33]:
print_report(sorted_results_evol)

Best architecture found after 5640 evaluated models!
Best model found: 


,id,dna,cell_spec,val_acc,latency,time_cost,total_time,test_acc,test_loss,test_per_time,test_all_time,time_spent_in_hours,time_spent_in_secs
5068,5069,"DNA([7, 7, 7, 6, 6])",64:64:64:56:56,85.02,0.020324,69.808378,323456.034085,93.18,0.261427,0.860657,77.459171,89,323456.034


In [34]:
sorted_results_evol.to_csv('data/evolution_100h.csv', index=False)

## Test with Reinforcement Learning Algorithm

In [77]:
sorted_results_rl = test_nas_algo('rl')
sorted_results_rl.head()

[2023-06-28 01:48:56] Try to use the default NATS-Bench (size) path from fast_mode=True and path=None.
Candidates: [8, 16, 24, 32, 40, 48, 56, 64]


ValueError: in user code:

    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1569 predict_function  *
        return step_function(self, iterator)
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/guilherme/data2/anaconda3/envs/icao_nets_training/lib/python3.8/site-packages/tensorflo

In [45]:
print_report(sorted_results_rl)

Best architecture found after 239 evaluated models!
Best model found: 


,id,dna,cell_spec,val_acc,latency,time_cost,total_time,test_acc,test_loss,test_per_time,test_all_time,time_spent_in_hours,time_spent_in_secs
229,230,"DNA([3, 4, 4, 3, 3])",32:40:40:32:32,82.74,0.01439,49.39539,10340.721351,91.67,0.297276,0.728126,65.531344,2,10340.721


In [46]:
sorted_results_rl.to_csv('data/rl_100h.csv', index=False)